In [1]:
from Scripts.SQLiteFun import execution, executionALL, create_sqlite_database
from hapiclient.util import HAPIError
from hapiclient import hapi
from time import sleep

def StartStopSampleCheck(prodKeys, conn):
    results = []
    for prodKey in prodKeys:
        multiKeys = False
        # check if multiple prodKeys for same URL (mult keys in one string)
        if ", " in prodKey:
            multiKeys = True
            index = prodKeys.index(prodKey)
            prodKey = prodKey.replace("\'", "")
            # keep separating them until each prodKey is in own string
            # while ", " in prodKey:
            before, sep, after = prodKey.partition(", ")
            prodKeys[index] = before # remove this line if need to check all keys
            #prodKeys[index] = after
            #prodKeys.insert(index, before)
            prodKey = prodKeys[index]
        dataset = str(prodKey)

        has_sampleStart = False
        has_sampleStop = False

        # initialize HAPI server
        server = 'https://cdaweb.gsfc.nasa.gov/hapi'

        # get row number of record updated for visual confirmation later
        if multiKeys:
            Record_id = execution(f""" SELECT rowNum 
                                    FROM (SELECT TestResults.rowNum, SPASE_id, prodKey, url FROM TestResults 
                                        INNER JOIN MetadataEntries USING (SPASE_id))
                                    WHERE prodKey LIKE '{dataset},%' AND url LIKE '%/hapi';""", conn)
        else:
            Record_id = execution(f""" SELECT rowNum 
                                FROM (SELECT TestResults.rowNum, SPASE_id, prodKey, url FROM TestResults 
                                    INNER JOIN MetadataEntries USING (SPASE_id))
                                WHERE prodKey = '{dataset}' AND url LIKE '%/hapi';""", conn)
        
        # get SPASE_id that matches the dataset
        if multiKeys:
            prodKeyStmt = f""" SELECT SPASE_id FROM MetadataEntries
                                WHERE prodKey LIKE '{dataset},%' """
        else:
            prodKeyStmt = f""" SELECT SPASE_id FROM MetadataEntries
                                WHERE prodKey = '{dataset}' """
        SPASE_ID = execution(prodKeyStmt, conn)
        try:
            SPASE_ID = SPASE_ID[0]
        # means that this dataset was not the only one for its record
        except IndexError:
            #print("IndexError occured at this dataset.")
            prodKeyStmt = f""" SELECT SPASE_id FROM MetadataEntries
                                WHERE prodKey LIKE '{dataset}%' """
            SPASE_ID = execution(prodKeyStmt, conn)
            SPASE_ID = SPASE_ID[0]
            Record_id = execution(f""" SELECT rowNum 
                                    FROM (SELECT TestResults.rowNum, SPASE_id, prodKey, url FROM TestResults 
                                        INNER JOIN MetadataEntries USING (SPASE_id))
                                    WHERE prodKey LIKE '{dataset}_,%' AND url LIKE '%/hapi';""", conn)
        
        # retrieve all parameters and the start date from the server
        sleep(5.0)
        try:
            meta = hapi(server,dataset)
        # catches errors like 'unknown dataset id'
        except HAPIError as e:
            #print("Caught HAPIError for dataset " + dataset + ", trying next.")
            #print(" ", e)
            continue
        else:
            # check for sample start and stop times
            for k, v in meta.items():
                if k == "sampleStartDate":
                    print(f"Dataset {dataset} has a sample start param") 
                    if v != None:
                        has_sampleStart = True
                        print(f"Dataset {dataset} has a sample start value") 
                elif k == "sampleStopDate":
                    print(f"Dataset {dataset} has a sample start param") 
                    if v != None:
                        has_sampleStop = True
                        print(f"Dataset {dataset} has a sample stop value") 
            if has_sampleStart and has_sampleStop:
                results.append(f"{SPASE_ID}, 1")
            else:
                results.append(f"{SPASE_ID}, 0")
    return results

# input abs path of database file you wish to query from
conn = create_sqlite_database("/home/jovyan/HDRL-Internship-2024/SPASE_Data_20240716.db")

haveBoth = 0
doNotHave = 0

HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE '%/hapi' LIMIT 5"""
prodKeys = execution(HapiStmt, conn)

results = StartStopSampleCheck(prodKeys, conn)

for result in results:
    if ", 1" in result:
        haveBoth += 1

# Note that if numbers are off slightly, this is because this does not keep track of how many failed info check
doNotHave = (len(results) - haveBoth)
print("The number of datasets that have sample start and stop times are: " + str(haveBoth))
print("The number of datasets that do not have them are: " + str(doNotHave))

The number of datasets that have sample start and stop times are: 0
The number of datasets that do not have them are: 5
